In [19]:
!unzip llama_1B_breast_cancer.zip -d llama_1B_breast_cancer

Archive:  llama_1B_breast_cancer.zip
  inflating: llama_1B_breast_cancer/llama_1B_breast_cancer/adapter_config.json  
  inflating: llama_1B_breast_cancer/llama_1B_breast_cancer/adapter_model.safetensors  
  inflating: llama_1B_breast_cancer/llama_1B_breast_cancer/README.md  
  inflating: llama_1B_breast_cancer/llama_1B_breast_cancer/special_tokens_map.json  
  inflating: llama_1B_breast_cancer/llama_1B_breast_cancer/tokenizer.json  
  inflating: llama_1B_breast_cancer/llama_1B_breast_cancer/tokenizer_config.json  


In [ ]:
import json
import re
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from peft import PeftModel, PeftConfig

import huggingface_hub
huggingface_hub.login("HF_TOKEN")

MODEL_NAME = "meta-llama/Llama-3.2-1B-Instruct"  # Update as needed
PEFT_PATH = "llama_1B_breast_cancer/llama_1B_breast_cancer"
DATASET_PATH = "pubmedqa_breast_cancer_mesh_subset.json"
DEVICE = 0  # CUDA device

with open(DATASET_PATH) as f:
    dataset = json.load(f)

examples = []
for qid, entry in dataset.items():
    question = entry["QUESTION"]
    context = " ".join(entry["CONTEXTS"]) if entry["CONTEXTS"] else ""
    prompt = (
        f"Context: {context}\nQuestion: {question} The answer is:"
    )
    gold = entry["final_decision"].strip().lower()
    examples.append({"id": qid, "prompt": prompt, "gold": gold})

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map="auto")
model = PeftModel.from_pretrained(model, PEFT_PATH)
qa_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

# def extract_final_answer(text):
#     match = re.search( r"\$\\boxed\{(yes|no|maybe)\}\$", text, re.IGNORECASE)
#     if match:
#         return match.group(1).lower()
#     return None

def extract_first_word(text):
    # Split text into words, strip leading/trailing whitespace
    words = text.strip().split()
    if words:
        # Remove punctuation if needed
        return re.sub(r"\W+$", "", words[0].lower())
    return None

results = []
correct = 0
total = 0

for ex in tqdm(examples):
    output = qa_pipeline(ex["prompt"], max_new_tokens=20)[0]["generated_text"]
    answer = extract_first_word(output[len(ex["prompt"]):])
    print(output)
    print(answer)
    results.append({"id": ex["id"], "gold": ex["gold"], "pred": answer, "raw_output": output})
    if answer != None and answer in ["yes", "no", "maybe"]:
        total += 1
    if answer == ex["gold"]:
        correct += 1

accuracy = correct / total
print(f"Accuracy: {accuracy:.4f}")

with open("pubmedqa_cardiology_llama3_eval_results_reasoning.json", "w") as f:
    json.dump(results, f, indent=2)

Device set to use cuda:0
  1%|          | 1/81 [00:00<00:37,  2.14it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Context: Telephone counseling and tailored print communications have emerged as promising methods for promoting mammography screening. However, there has been little research testing, within the same randomized field trial, of the efficacy of these two methods compared to a high-quality usual care system for enhancing screening. This study addressed the question: Compared to usual care, is tailored telephone counseling more effective than tailored print materials for promoting mammography screening? Three-year randomized field trial. One thousand ninety-nine women aged 50 and older recruited from a health maintenance organization in North Carolina. Women were randomized to 1 of 3 groups: (1) usual care, (2) tailored print communications, and (3) tailored telephone counseling. Adherence to mammography screening based on self-reports obtained during 1995, 1996, and 1997. Compared to usual care alone, telephone counseling promoted a significantly higher proportion of women having mammogra

  2%|▏         | 2/81 [00:00<00:34,  2.29it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Context: Lymphedema may be identified by simpler circumference changes as compared with changes in limb volume. Ninety breast cancer patients were prospectively enrolled in an academic trial, and seven upper extremity circumferences were measured quarterly for 3 years. A 10% volume increase or greater than 1 cm increase in arm circumference identified lymphedema with verification by a lymphedema specialist. Sensitivity and specificity of several different criteria for detecting lymphedema were compared using the academic trial as the standard. Thirty-nine cases of lymphedema were identified by the academic trial. Using a 10% increase in circumference at two sites as the criterion, half the lymphedema cases were detected (sensitivity 37%). When using a 10% increase in circumference at any site, 74.4% of cases were detected (sensitivity 49%). Detection by a 5% increase in circumference at any site was 91% sensitive.
Question: Can a practicing surgeon detect early lymphedema reliably? The

  4%|▎         | 3/81 [00:01<00:33,  2.36it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Context: Retrospective outcome measurement study. The purpose of this study is to assess whether ossification of the posterior longitudinal ligament (OPLL) affects neurologic outcomes in patients with acute cervical spinal cord injury (SCI). There have so far been few reports examining the relationship between OPLL and SCI and there is controversy regarding the deteriorating effects of OPLL-induced canal stenosis on neurologic outcomes. To obtain a relatively uniform background, patients nonsurgically treated for an acute C3-C4 level SCI without any fractures or dislocations of the spinal column were selected, resulting in 129 patients. There were 110 men and 19 women (mean age was 61.1 years), having various neurologic conditions on admission (American Spinal Injury Association [ASIA] impairment scale A, 43; B, 16; C, 58; D, 12). The follow-up period was the duration of their hospital stay and ranged from 50 to 603 days (mean, 233 days). The presence of OPLL, the cause of injury, the 

 99%|█████████▉| 80/81 [00:32<00:00,  2.32it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Context: To assess whether eligibility to an adjuvant chemotherapy protocol in itself represents a good prognostic factor after radical cystectomy for bladder cancer. Between April 1984 and May 1989, our institution entered 35 patients with invasive bladder cancer into the Swiss Group for Clinical and Epidemiological Cancer Research (SAKK) study 09/84. They were randomly assigned to either observation or three postoperative courses of cisplatin monotherapy after cystectomy. This study had a negative result. The outcome of these 35 patients (protocol group) was compared with an age- and tumor-stage-matched cohort (matched group; n = 35) who also underwent cystectomy during the same period, but were not entered into the SAKK study, as well as the remaining 57 patients treated during the study period for the same indication (remaining group). Median overall survival decreased from 76.3 months in the protocol group to 52.1 months in the matched group and to 20.3 months in the remaining gro

100%|██████████| 81/81 [00:32<00:00,  2.48it/s]

Context: Currently the choice of breast cancer therapy is based on prognostic factors. The proliferation marker Ki-67 is used increasingly to determine the method of therapy. The current study analyses the predictive value of Ki-67 in foreseeing breast cancer patients' responses to neoadjuvant chemotherapy. This study includes patients with invasive breast cancer treated between 2008 and 2013. The clinical response was assessed by correlating Ki-67 to histological examination, mammography, and ultrasonography findings. The average Ki-67 value in our patients collectively (n = 77) is 34.9 ± 24.6%. The average Ki-67 value is the highest with 37.4 ± 24.0% in patients with a pCR. The Ki-67 values do not differ significantly among the 3 groups: pCR versus partial pathological response versus stable disease/progress (P = 0.896). However, Ki-67 values of patients with luminal, Her2 enriched, and basal-like cancers differed significantly from each other. Furthermore, within the group of lumina